This notebook shows how to load data exported from QuPath, compute metrics and display them, according to the configuration file. This is meant for a single-animal.

There are some conventions that need to be met in the QuPath project so that the measurements are usable with `histoquant`:
+ Objects' classifications must be derived, eg. be in the form "something: else". The primary classification ("something") will be refered to "object_type" and the secondary classification ("else") to "channel" in the configuration file.
+ Only one "object_type" can be processed at once, but supports any numbers of channels.
+ Annotations (brain regions) must have properly formatted measurements. For punctual objects, it would be the count. Run the "add_regions_count.groovy" script to add them. The measurements names must be in the form "something: else name", for instance, "something: else Count". "name" is refered to "base_measurement" in the configuration file.

You should copy this notebook, the configuration file and the atlas-related configuration files (blacklist and fusion) elsewhere and edit them according to your need.

The data was generated from QuPath with stardist cell detection on toy data.

In [ ]:
import pandas as pd

import histoquant as hq

In [ ]:
# Full path to your configuration file, edited according to your need beforehand
config_file = "../../resources/demo_config_cells.toml"

In [ ]:
# - Files
# animal identifier
animal = "animalid0"
# set the full path to the annotations tsv file from QuPath
annotations_file = "../../resources/cells_measurements_annotations.tsv"
# set the full path to the detections tsv file from QuPath
detections_file = "../../resources/cells_measurements_detections.tsv"

In [ ]:
# get configuration
cfg = hq.config.Config(config_file)

In [ ]:
# read data
df_annotations = pd.read_csv(annotations_file, index_col="Object ID", sep="\t")
df_detections = pd.read_csv(detections_file, index_col="Object ID", sep="\t")

# remove annotations that are not brain regions
df_annotations = df_annotations[df_annotations["Classification"] != "Region*"]
df_annotations = df_annotations[df_annotations["ROI"] != "Rectangle"]

# convert atlas coordinates from mm to microns
df_detections[["Atlas_X", "Atlas_Y", "Atlas_Z"]] = df_detections[
    ["Atlas_X", "Atlas_Y", "Atlas_Z"]
].multiply(1000)

# have a look
display(df_annotations.head())
display(df_detections.head())

In [ ]:
# get distributions per regions, spatial distributions and coordinates
df_regions, dfs_distributions, df_coordinates = hq.process.process_animal(
    animal, df_annotations, df_detections, cfg, compute_distributions=True
)

# have a look
display(df_regions.head())
display(df_coordinates.head())

In [ ]:
# plot distributions per regions
figs_regions = hq.display.plot_regions(df_regions, cfg)
# specify which regions to plot
# figs_regions = hq.display.plot_regions(df_regions, cfg, names_list=["GRN", "IRN", "MDRNv"])

# save as svg
# figs_regions[0].savefig(r"C:\Users\glegoc\Downloads\regions_count.svg")
# figs_regions[1].savefig(r"C:\Users\glegoc\Downloads\regions_density.svg")

In [ ]:
# plot 1D distributions
fig_distrib = hq.display.plot_1D_distributions(
    dfs_distributions, cfg, df_coordinates=df_coordinates
)

If there were several `animal` in the measurement file, it would be displayed as mean +/- sem instead.

In [ ]:
# plot heatmap (all types of cells pooled)
fig_heatmap = hq.display.plot_2D_distributions(df_coordinates, cfg)